In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData-master/DefectData/inst/extdata/terapromise/mccabe/PC4.arff')
df = pd.DataFrame(data[0])

In [2]:
df.shape

(1287, 38)

In [3]:
df.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,8.0,1.0,0.0,0.0,6.0,0.0,1.0,0.07,0.0,0.00,...,2.0,0.03,28.0,43.0,2.0,4.0,30.0,28.57,15.0,b'N'
1,1.0,7.0,3.0,0.0,0.0,0.0,4.0,0.44,0.0,0.00,...,14.0,0.25,17.0,45.0,10.0,12.0,16.0,0.00,9.0,b'N'
2,33.0,29.0,7.0,9.0,20.0,52.0,15.0,0.25,24.0,2.17,...,54.0,0.13,104.0,168.0,29.0,25.0,115.0,36.25,60.0,b'N'
3,1.0,19.0,0.0,17.0,0.0,0.0,10.0,0.21,0.0,0.00,...,32.0,0.20,102.0,141.0,28.0,26.0,50.0,35.42,48.0,b'N'
4,7.0,13.0,5.0,9.0,9.0,24.0,7.0,0.22,12.0,2.00,...,25.0,0.14,37.0,74.0,15.0,13.0,50.0,43.90,32.0,b'Y'


In [4]:
buggy = b'Y'
clean = b'N'
x = df.drop(['Defective'],axis = 'columns')

for i in range(1287):
    if df.iloc[i,37] == buggy:
        df.iloc[i,37] = 1
    else:
        df.iloc[i,37] = 0

y = df['Defective']
y = y.astype(int)
y.head()

0    0
1    0
2    0
3    0
4    1
Name: Defective, dtype: int32

In [5]:
buggy = 1
clean = 0

In [6]:
from sklearn.cluster import AffinityPropagation

af_model = AffinityPropagation(damping=0.9)
af_model.fit(x)
af_prediction = af_model.predict(x)

af_prediction

array([47,  6,  0, ..., 47, 17, 27], dtype=int64)

In [7]:
clusters = unique(af_prediction)
print(clusters)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]


In [8]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if af_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if af_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]


In [11]:
TP = 1
TN = 1
FP = 1
FN = 1

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == af_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient
MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.06336148864298564
Accuracy : 0.8520526723470179
F-Score : 0.07729468599033816


In [12]:
from sklearn.cluster import OPTICS

optics_model = OPTICS()
optics_prediction = optics_model.fit_predict(x)

optics_prediction[:5]

array([-1, -1, -1, 80, 49])

In [13]:
clusters = unique(optics_prediction)
print(clusters)

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70
 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90]


In [14]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if optics_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if optics_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [15]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == optics_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.10301923969693327
Accuracy : 0.7987567987567987
F-Score : 0.21752265861027192


In [16]:
from sklearn.cluster import MeanShift

MeanShift_model = MeanShift()
MeanShift_prediction = MeanShift_model.fit_predict(x)

clusters = unique(MeanShift_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if MeanShift_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if MeanShift_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 1
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == MeanShift_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.1115157113228866
Accuracy : 0.8633540372670807
F-Score : 0.05376344086021506


In [17]:
from sklearn.cluster import DBSCAN

DBSCAN_model = DBSCAN()
DBSCAN_prediction = DBSCAN_model.fit_predict(x)

clusters = unique(DBSCAN_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if DBSCAN_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if DBSCAN_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == DBSCAN_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1

TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.011126032654836467
Accuracy : 0.13819875776397517
F-Score : 0.24180327868852458


In [18]:
from pyclustering.cluster.xmeans import xmeans

xmeans_instance = xmeans(x)
xmeans_instance.process()
groups = xmeans_instance.get_clusters()

xmeans_prediction = [0]*len(y)

for i in range(len(groups)):
    for u in groups[i]:
        xmeans_prediction[u] = i

clusters = unique(xmeans_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if xmeans_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if xmeans_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == xmeans_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.06173827516781472
Accuracy : 0.8585858585858586
F-Score : 0.05208333333333333
